## GENERATE THE DATA

In [ ]:
import numpy, os, sys, pickle

In [ ]:
## create a dimension for each parameter
default_10dim_param_space = numpy.random.random((10,5))
print(default_10dim_param_space)
for row in range(len(default_10dim_param_space)):
    #print(row)
    if row >= 4:
        #print(default_10dim_param_space[row])
        default_10dim_param_space[row]=numpy.zeros(5)
    elif row >= 2:
        default_10dim_param_space[row]=default_10dim_param_space[1]
print()
print(default_10dim_param_space)

Here we generate the data by simulating the model:

In [ ]:
cwd = os.getcwd()
os.chdir("..")
sys.path.append(os.getcwd())
os.chdir(cwd)
from src.mc_prism import call_prism

In [ ]:
cwd

In [ ]:
from src.generate_data import generate_experiments_n_data,generate_experiments,generate_data

In [ ]:
def generate_experiments_n_data(model_types, multiparam, n_samples, populations, dimension_sample_size,
                                modular_param_space=None, debugging=False):
    """Generate experiment data for given settings

    Args
    ------
    model_types: list of model types
    multiparam: yes if multiparam should be used
    n_samples: list of sample sizes
    populations: list of agent populations
    dimension_sample_size: number of samples of in each paramter dimension to be used
    modular_param_space: parameter space to be used
    """
    max_sample = max(n_samples)
    start_time = time.time()

    i = 1
    Experiments = {}
    Data = {}
    for model_type in model_types:
        if multiparam:
            model_type = "multiparam_" + model_type
        if debugging:
            print("model_type",model_type)
        if "synchronous" in model_type:
            sim_lenght = 2
        Data[model_type] = {}
        Experiments[model_type] = {}
        for N in populations:
            if "semisynchronous" in model_type:
                sim_lenght = 2 * N
            if "asynchronous" in model_type:
                sim_lenght = 2 * N
            parameters = ["p"]
            if multiparam:
                for agents in range(1, N):
                    parameters.append("q" + str(agents))
            else:
                parameters.append("q")
            print("parameters: ", parameters)

            ## modulate parameter space
            if modular_param_space is not None:
                param_space = modular_param_space
            else:
                param_space = numpy.random.random((len(parameters), dimension_sample_size))

            print("parameter space:")
            print(param_space)

            model = model_type + str(N) + ".pm"

            Experiments[model_type][N] = {}
            Data[model_type][N] = {}
            for n_sample in n_samples:
                Experiments[model_type][N][n_sample] = {}
                Data[model_type][N][n_sample] = {}

            #print(len(param_space[0]))
            for column in range(len(param_space[0])):
                column_values = []
                for value in param_space[:, column]:
                    column_values.append(value)
                column_values = tuple(column_values)
                print("parametrisation:",column_values)
                for n_sample in n_samples:
                    Experiments[model_type][N][n_sample][column_values] = []
                    Data[model_type][N][n_sample][column_values] = []
                # file = open("path_{}_{}_{}_{}_{}.txt".format(model_type,N,max_sample,v_p,v_q),"w+")
                # file.close()
                for sample in range(1, max_sample + 1):
                    ## dummy path file for prism output
                    parameter_values = ""
                    prism_parameter_values = ""

                    for value in range(len(column_values)):
                        parameter_values = parameter_values + "_" + str(column_values[value])
                        prism_parameter_values = prism_parameter_values + str(parameters[value]) + "=" + str(
                            column_values[value]) + ","
                    prism_parameter_values = prism_parameter_values[:-1]
                    # print(parameter_values)
                    # print(prism_parameter_values)

                    ## more profound path name here
                    ## path_file = f"path_{model_type}{N}_{max_sample}_{parameter_values}.txt"
                    path_file = "dummy_path" + str(time.time()) + ".txt"
                    # print(path_file)
                    if debugging:
                        print(f"{model} -const {prism_parameter_values} -simpath {str(sim_lenght)} {path_file}")

                    ## here is the PRISM called
                    call_prism(f"{model} -const {prism_parameter_values} -simpath {str(sim_lenght)} {path_file}",
                               silent=True, prism_output_path=cwd, std_output_path=None)
                    ## parse the dummy file
                    file = open(path_file, "rt")
                    last_line = file.readlines()[-1]

                    ## close dummy file
                    file.close()

                    ## append the experiment
                    state = sum(list(map(lambda x: int(x), last_line.split(" ")[2:-1])))

                    ## some error occured
                    if state > N or debugging or "2" in last_line.split(" ")[2:-1]:
                        print(last_line[:-1])
                        print("state:", state)
                        print()
                    else:  ## if no error remove the file
                        os.remove(path_file)
                    for n_sample in n_samples:
                        if sample <= n_sample:
                            Experiments[model_type][N][n_sample][column_values].append(state)
                for n_sample in n_samples:
                    for i in range(N + 1):
                        Data[model_type][N][n_sample][column_values].append(len(list(
                            filter(lambda x: x == i, Experiments[model_type][N][n_sample][column_values]))) / n_sample)
                print("states:", Experiments[model_type][N][max_sample][column_values])

    print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")
    return Experiments, Data


def generate_experiments(model_types, multiparam, n_samples, populations, dimension_sample_size,
                         modular_param_space=None):
    """Generate experiment data for given settings
    Args
    ------
    model_types: list of model types
    multiparam: yes if multiparam should be used
    n_samples: list of sample sizes
    populations: list of agent populations
    dimension_sample_size: number of samples of in each paramter dimension to be used
    modular_param_space: parameter space to be used
    """
    return generate_experiments_n_data(model_types, multiparam, n_samples, populations, dimension_sample_size,
                                       modular_param_space)[0]


def generate_data(model_types, multiparam, n_samples, populations, dimension_sample_size, modular_param_space=None):
    """Generate experiment data for given settings

    Args
    ------
    model_types: list of model types
    multiparam: yes if multiparam should be used
    n_samples: list of sample sizes
    populations: list of agent populations
    dimension_sample_size: number of samples of in each paramter dimension to be used
    modular_param_space: parameter space to be used
    """
    return generate_experiments_n_data(model_types, multiparam, n_samples, populations, dimension_sample_size,
                                       modular_param_space)[1]

## two-param

In [ ]:
multiparam = False
model_types = ["synchronous_parallel_"]
n_samples = [3500,1500,100]
populations = [2,3,5,10]
dimension_sample_size = 5

In [ ]:
#p_values=[]
#for v_p in numpy.random.uniform(0.0, 1.0,dimension_sample_size):
#    p_values.append(v_p)
#q_values=[]
#for v_q in numpy.random.uniform(0.0, 1.0,dimension_sample_size):
#    q_values.append(v_q)

In [ ]:
p_values = [0.028502714675268215, 0.45223461506339047, 0.8732745414252937, 0.6855555397734584, 0.13075717833714784]
q_values = [0.5057623641293089  , 0.29577906622244676, 0.8440550299528644, 0.8108008054929994, 0.03259111103419188]

In [ ]:
default_2dim_param_space = numpy.zeros((2,5))
default_2dim_param_space[0] = p_values
default_2dim_param_space[1] = q_values
default_2dim_param_space

In [ ]:
# Experiments_two_param, Data_two_param = generate_experiments_n_data(model_types,multiparam,n_samples,populations,dimension_sample_size,default_2dim_param_space)

This took Freya 6 days to run

As you can see this process took a while so we have pickled the data and we load it back

In [ ]:
# pickle.dump( Experiments_two_param,open(os.path.join(data_path,"Experiments_two_param.p", "wb" )))
# pickle.dump(Data_two_param,open(os.path.join(data_path,"Data_two_param.p"), "wb"))

In [ ]:
Experiments_two_param = pickle.load(open(os.path.join(data_path,"Experiments_two_param.p"), "rb" ))
Data_two_param = pickle.load(open(os.path.join(data_path,"Data_two_param.p"), "rb" ))

## multiparam

## Experiments_one_point_step_down_10dim_param_space

In [ ]:
multiparam = True
model_types = ["synchronous_parallel_"]
n_samples = [3500,1500,100]
populations = [10]
dimension_sample_size = 5

In [ ]:
one_point_step_down_10dim_param_space = numpy.zeros((10,1))
for i in range(0,4):
    one_point_step_down_10dim_param_space[i] = [0.1]
for i in range(4,10):
    one_point_step_down_10dim_param_space[i] = [0]
one_point_step_down_10dim_param_space

In [ ]:
# Experiments_one_point_step_down_10dim_param_space, Data_one_point_step_down_10dim_param_space = generate_experiments_n_data(model_types,multiparam,n_samples,populations,dimension_sample_size,one_point_step_down_10dim_param_space)

In [ ]:
#pickle.dump( Experiments_one_point_step_down_10dim_param_space,open(os.path.join(data_path,"Experiments_one_point_step_down_10dim_param_space.p"), "wb" ))
#pickle.dump( Data_one_point_step_down_10dim_param_space,open(os.path.join(data_path,"Data_one_point_step_down_10dim_param_space.p"), "wb" ))

In [ ]:
Experiments_one_point_step_down_10dim_param_space = pickle.load(open(os.path.join(data_path,"Experiments_one_point_step_down_10dim_param_space.p"), "rb" ))
Data_one_point_step_down_10dim_param_space = pickle.load(open(os.path.join(data_path,"Data_one_point_step_down_10dim_param_space.p"), "rb" ))

## linear generations TBD

In [ ]:
multiparam = True
model_types = ["synchronous_parallel_"]
n_samples = [3500,1500,100]
populations = [10]
dimension_sample_size = 5

In [ ]:
one_point_linear_10dim_param_space  = numpy.zeros((10,1))
one_point_linear_10dim_param_space[0] = [0.1]
for i in range(1,10):
    one_point_linear_10dim_param_space[i] = [0.1 + i*0.5]
one_point_linear_10dim_param_space